In [2]:
%display latex

<h1><p style="color:#16A085"> Álgebra Computacional </p style></h1>
<h2> Índice </h2>

- 1. Algoritmo de Euclides
    - 1.1 Algoritmo de Euclides para un Dominio Euclídeo
    - 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo
    - 1.3 Ejemplos en $\mathbb{Z}$
    - 1.4 Ejemplos en $\mathbb{Q}[x]$
    - 1.5 Ejemplos en $\mathbb{R}[x]$
    - 1.6 Ejemplos en $\mathbb{Z}[i]$
    

# 1. Algoritmo de Euclides

# 1.1 Algoritmo de Euclides para un Dominio Euclídeo

In [40]:
def euclides(a, b, mod):
    if (b == 0):
        return a
    else:
        return euclides (b, (mod(a, b)), mod)

## 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo

In [41]:
def extendedEuclides(a, b, div_mod):
    if (b == 0):
        return a, 1, 0 
    else:
        q, r = div_mod(a, b)
        gcd, d, e = extendedEuclides(b, r, div_mod)
        return gcd, e, (d -q * e)

## 1.3 Ejemplos en $\mathbb{Z}$

In [42]:
def mod_int(a, b): return a % b
def div_mod_int(a, b): return a // b, a % b

print(euclides(55, 22, mod_int))
print(extendedEuclides(55, 22, div_mod_int))

11
(11, 1, -2)


## 1.4 Ejemplos en $\mathbb{Q}[x]$

In [43]:
def div_mod_pol(a, b): return a.quo_rem(b);
def mod_pol(a, b): _, r =  div_mod_pol(a, b); return r
    
R.<x> = QQ[]

a = (x^2 +2*x + 1)
b =  (x^2 - 1)

print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-3/2)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol)).monic()

2*x + 2
(2*x + 2, 1, -1)
x^3 + 2/3*x^2 + x + 2/3


# 2. Teorema chino del resto

In [67]:
def chineseRemainder(coprimes, elems, div_mod, rem):
    m = 1
    for mi in coprimes:
        m = m * mi
    result = 0;
    for i in range(len(coprimes)):
        n = m / coprimes[i]
        _, a, _ = extendedEuclides(n, coprimes[i], div_mod)
        c = rem((a * elems[i]), coprimes[i])
        result = c * n + result
    return result

## 2.1 Ejemplos en $\mathbb{Z}$

In [68]:
def rem(a, b):
    r = abs(a) % abs(b)
    return r if a >= 0 else -r
chineseRemainder([3,5,7],[4,5,33], div_mod_int, rem)

40

# 3. MCD en DFU

In [10]:
def gcdDFU(a, b):
    if a.degree() < b.degree():
        return mcdDFU(b,a)
    while b != 0:
        q,r = (b.leading_coefficient()**(a.degree() - b.degree() + 1) * a).quo_rem(b)
        a = b
        b = r
    return a

## 3.1 Ejemplos en $\mathbb{Z}[x]$

In [11]:
R.<x> = ZZ[]
a = (x + 1)**2
b = (x + 1)*(x - 1)
gcdDFU(a, b).monic()

x + 1

# 4. Inverso de un elemento en un cuerpo finito

In [12]:
def inverseFiniteField(a, field, div_mod):
    _, _, b = extendedEuclides(field.cardinality(), a, div_mod)
    return field(b)

## 4.1 Ejemplos en $\mathbb{Z}_p$

In [13]:
Z7 = Integers(7)
a = 3
print(inverseFiniteField(a, Z7, div_mod_int))

Z31 = Integers(31)
b = 14
print(inverseFiniteField(b, Z31, div_mod_int))

Z61 = Integers(61)
c = 27
print(inverseFiniteField(c, Z61, div_mod_int))

5
20
52


# 5. Test de irreducibilidad de un polinomio en $\mathbb{F}_q[x]$

In [75]:
def fast_exp_mod(b,e,m):
#Devuelve b^e mod m.
    r = 1
    b = b%m
    while e>0:
        if e%2 == 1:
            r = (r*b)%m
        e = e//2
        b  = (b*b)%m
    return r 

def irreducibility_test(f,q):
    h = (x) % f
    l = f.degree()
    for k in range(1, l // 2 + 1):
        h = fast_exp_mod(h,q,f)
        if gcd(h-x, f) != 1: return False
    return True

In [81]:
R.<x>=PolynomialRing(GF(7))
f = x^21+x^2+1
print "El polinomio ", f, " es irreducible en F2[x]?: ", irreducibility_test(f,2), " ", f.is_irreducible()
g = R.random_element(50)
print "El polinomio ", g, " es irreducible en F2[x]?: ", irreducibility_test(g,2), " ", g.is_irreducible()

El polinomio  x^21 + x^2 + 1  es irreducible en F2[x]?:  True   False
El polinomio  3*x^50 + 6*x^49 + x^48 + 2*x^47 + 6*x^46 + 4*x^45 + 2*x^44 + 2*x^43 + 2*x^42 + 3*x^41 + 2*x^40 + 6*x^39 + 5*x^38 + x^37 + 4*x^36 + 2*x^35 + 2*x^33 + 2*x^32 + 6*x^31 + 4*x^30 + 3*x^29 + 5*x^28 + 5*x^27 + x^26 + x^25 + x^24 + 3*x^23 + 2*x^21 + 3*x^20 + 6*x^19 + x^17 + 6*x^15 + 2*x^14 + 2*x^13 + 3*x^12 + 3*x^10 + 5*x^9 + 2*x^8 + 6*x^7 + 5*x^6 + 4*x^5 + 5*x^4 + 6*x^3 + 2*x^2 + 5*x + 5  es irreducible en F2[x]?:  True   False


In [82]:
F= factor(2016)
primes =[f[0] for f in F for i in range(f[1])]
for k in primes:
    print k

2
2
2
2
2
3
3
7


# 6. Logaritmo discreto en cuerpos $\mathbb{F}_q[x]/(f(x))$